In [1]:
import json
from pymongo import MongoClient

In [20]:
data_loc = "../result/inserted/2/quora/"

data_name = "quora_last/k4/results quorak4 100 75 lrrs.json"
client = MongoClient('localhost', 27017)
db = client['thesis']
result = db.quorak4

### storing in mongodb

In [21]:
with open(data_name, 'r') as result_file:
    result_json = json.load(result_file)
    for document in result_json["results"]:
        #remove predict and true value beacuase it creates DocumentTooLarge Error since one document is above 16MB
#         del document["train_true_predict"]
#         del document["test_true_predict"]
        result.insert_one(document)

### Reading json for validation and checking

In [25]:
with open(data_loc+data_name+".json") as json_file: 
    data = json.load(json_file)
    
type(data["results"])
len(data["results"])

3

In [29]:
for i in data["results"]:
    print(i["dataset"],i["algorithm"], i["k_fold"],i["d2v_vec_size"], i["min_leaf_point"],i["epochs"], i["feature_size"])

quora lr_mvdt 3 25 5 100 2
quora lr_mvdt 3 25 5 100 5
quora lr_mvdt 3 25 5 100 10


In [ ]:
data["results"]

### Accuracy vs depth

In [3]:
dataset = '20ng_CG_RM'
vector_sizes_list = [10, 25, 50, 75, 100]
algorithms_list = ['lr_mvdt', 'rs_mvdt']
k = 5
epochs_list = [100, 300, 500, 800, 1000]
n_features_list = [2, 5, 10, 20, all]
min_leaf_point_list = [5, 10, 15, 20, 30]

In [38]:
by_list = [['by_n_features', '$feature_size'],
           ['by_min_leaf_points', '$min_leaf_point'],
           ['by_epochs', '$epochs'],
           ['by_vectors', '$d2v_vec_size'],
           ['by_k_folds', '$k_fold'],
           ['by_datasets', '$algorithm']]

In [14]:
for vector_size in vector_sizes_list:
    for algorithm in algorithms_list:    
        calculate(vector_size, algorithm)
        print(vector_size, algorithm)

10 lr_mvdt
10 rs_mvdt
25 lr_mvdt
25 rs_mvdt
50 lr_mvdt
50 rs_mvdt
75 lr_mvdt
75 rs_mvdt
100 lr_mvdt
100 rs_mvdt


In [13]:
def calculate(vector_size, algorithm):    
    query = [{'$match': {'dataset': dataset, 'd2v_vec_size': vector_size, 'algorithm': algorithm}},
         {'$project':{'intermediate_result': 1}},        
         ]
    results = db['20ng_CG_RM'].aggregate(query)

    for result in results:        
        for i in result['intermediate_result']:
            if i is not None:
                new_result = {'dataset': dataset,
                              'd2v_vec_size': vector_size,
                              'algorithm': algorithm,                          
                              'on_depth': i['on_depth'],
                              'train_acc': i['accuracy'][0],
                              'test_acc': i['accuracy'][1]
                             }
                
                db.acc_vs_depth_ng.insert_one(new_result)